In [0]:
import pandas as pd 
#import findspark
#findspark.init('/opt/spark')
import pyspark
import random
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf 
from pyspark.sql.types import StructField,IntegerType, StructType,StringType, FloatType
from pyspark.sql.functions import desc 
from pyspark.sql.functions import asc 
import pyspark.sql.functions as F 
import numpy as np
from pyspark.sql import Window
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

In [0]:
%sql
Show databases;
use databricks_metastore


In [0]:
%python

registry = spark.sql("select * from databricks_metastore.nbg_enrollment_metrics_ban2401g000303_registry")
studysum = spark.sql("select * from databricks_metastore.nbg_enrollment_metrics_ban2401g000303_studysum")
a3blinded_prod = spark.sql("select * from databricks_metastore.nbg_enrollment_metrics_ban2401g000303_a3blinded_prod_20220105020004")
a45blinded_prod = spark.sql("select * from databricks_metastore.nbg_enrollment_metrics_ban2401g000303_a45blinded_prod_20220105023004")






In [0]:
df = spark.sql('select * from databricks_metastore.nbg_enrollment_metrics_ban2401g000303_registry limit 100')
df.display()

subject_event_crf_id,user_create_id,user_created_username,user_created_fullname,ts_create,user_last_modify_id,user_last_modify_username,user_last_modify_fullname,ts_last_modify,dd_version_id,dd_version_label,dd_revision_id,site_id,site_code,site_initials,site_label,subject_id,subject_code,subject_label,event_id,event_code,event_label,event_is_unscheduled,language_code,language_label,has_unresolved_error,is_locked,user_last_lock_id,user_last_lock_username,user_last_lock_fullname,ts_last_lock,monitor_review,monitor_review_label,monitor_review_location,monitor_review_location_label,user_last_monitor_review_id,user_last_monitor_review_username,user_last_monitor_review_fullname,ts_last_monitor_review,dd_field_name,dd_revision_field_id,dd_revision_field_value,dd_revision_field_translated_value,dd_revision_field_is_missing,dd_revision_field_is_phi,dd_revision_field_label,dd_revision_field_unit,ingest_filename
118823,393,mark.dobrina,Mark Dobrina,2021-12-01 22:25:19.796662+00:00,393,mark.dobrina,Mark Dobrina,2021-12-01 22:25:19.796662+00:00,33,default,87,43,324,SMVH,Sharp Mesa Vista Hospital,1531,ahd324-11514,AHD324-11514,81,sc5,Screening - Stage 5,false,e,English,false,null,null,null,null,null,review_with_sdv,Review with SDV,remote,Remote,347,ebartolo,Earle Bartolome,2021-12-07 23:30:20.298320+00:00,comm,1573,null,null,false,false,Comments,null,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,33,default,87,58,41,YALE,Yale University School of Medicine,982,ahd041-10968,AHD041-10968,6,sc2,Screening - Stage 2,false,e,English,false,null,null,null,null,null,null,null,null,null,null,null,null,null,done,1565,1,Done,false,false,Visit,null,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,33,default,87,58,41,YALE,Yale University School of Medicine,982,ahd041-10968,AHD041-10968,6,sc2,Screening - Stage 2,false,e,English,false,null,null,null,null,null,null,null,null,null,null,null,null,null,ndreason,1566,null,null,false,false,"If Not Done, reason not done",null,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,33,default,87,58,41,YALE,Yale University School of Medicine,982,ahd041-10968,AHD041-10968,6,sc2,Screening - Stage 2,false,e,English,false,null,null,null,null,null,null,null,null,null,null,null,null,null,covreason,1567,null,null,false,false,Is the region around the Site affected by COVID-19 outbreak?,null,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,33,default,87,58,41,YALE,Yale University School of Medicine,982,ahd041-10968,AHD041-10968,6,sc2,Screening - Stage 2,false,e,English,false,null,null,null,null,null,null,null,null,null,null,null,null,null,examdate,1568,2021-09-10,2021-09-10,false,false,Visit Date,YYYY-MM-DD,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,1089,alyssa.miller,Alyssa Miller,2021-12-01 23:03:24.027015+00:00,33,default,87,58,41,YALE,Yale University School of Medicine,982,ahd041-10968,AHD041-10968,6,sc2,Screening - Stage 2,false,e,English,false,null,null,null,null,null,null,null,null,null,null,null,null,null,transfer,1569,1,Visit conducted at Home Site,false,false,Indicate if visit is related to a Permanent or Temporary Transfer,null,dbfs:/mnt/freezer/nbg/enrollment_metrics/ban2401g000303/registry/registry.csv
118854,1089,alyssa.miller,Alyssa Miller,2021-12-0

In [0]:
# show the column names

registry.columns

# getting the list of column names
#col = registry.columns
 
# printing
#print(f'List of column names: {col}')
 
# visualizing the dataframe
#registry.show()

Out[228]: ['subject_event_crf_id',
 'user_create_id',
 'user_created_username',
 'user_created_fullname',
 'ts_create',
 'user_last_modify_id',
 'user_last_modify_username',
 'user_last_modify_fullname',
 'ts_last_modify',
 'dd_version_id',
 'dd_version_label',
 'dd_revision_id',
 'site_id',
 'site_code',
 'site_initials',
 'site_label',
 'subject_id',
 'subject_code',
 'subject_label',
 'event_id',
 'event_code',
 'event_label',
 'event_is_unscheduled',
 'language_code',
 'language_label',
 'has_unresolved_error',
 'is_locked',
 'user_last_lock_id',
 'user_last_lock_username',
 'user_last_lock_fullname',
 'ts_last_lock',
 'monitor_review',
 'monitor_review_label',
 'monitor_review_location',
 'monitor_review_location_label',
 'user_last_monitor_review_id',
 'user_last_monitor_review_username',
 'user_last_monitor_review_fullname',
 'ts_last_monitor_review',
 'dd_field_name',
 'dd_revision_field_id',
 'dd_revision_field_value',
 'dd_revision_field_translated_value',
 'dd_revision_field_is_missing',
 'dd_revision_field_is_phi',
 'dd_revision_field_label',
 'dd_revision_field_unit',
 'ingest_filename']

In [0]:
studysum.columns

Out[229]: ['subject_event_crf_id',
 'user_create_id',
 'user_created_username',
 'user_created_fullname',
 'ts_create',
 'user_last_modify_id',
 'user_last_modify_username',
 'user_last_modify_fullname',
 'ts_last_modify',
 'dd_version_id',
 'dd_version_label',
 'dd_revision_id',
 'site_id',
 'site_code',
 'site_initials',
 'site_label',
 'subject_id',
 'subject_code',
 'subject_label',
 'event_id',
 'event_code',
 'event_label',
 'event_is_unscheduled',
 'language_code',
 'language_label',
 'has_unresolved_error',
 'is_locked',
 'user_last_lock_id',
 'user_last_lock_username',
 'user_last_lock_fullname',
 'ts_last_lock',
 'monitor_review',
 'monitor_review_label',
 'monitor_review_location',
 'monitor_review_location_label',
 'user_last_monitor_review_id',
 'user_last_monitor_review_username',
 'user_last_monitor_review_fullname',
 'ts_last_monitor_review',
 'dd_field_name',
 'dd_revision_field_id',
 'dd_revision_field_value',
 'dd_revision_field_translated_value',
 'dd_revision_field_is_missing',
 'dd_revision_field_is_phi',
 'dd_revision_field_label',
 'dd_revision_field_unit',
 'ingest_filename']

In [0]:
a3blinded_prod.columns




#https://medium.com/@zhlli/data-wrangling-pandas-vs-pyspark-dataframe-d625b0b99c73

Out[230]: ['participant_id',
 'ingest_filename',
 'region',
 'country',
 'site_id',
 'randomization_date',
 'apoe4_carrier',
 'trial_assignment',
 'weight',
 'disposition']

In [0]:

a45blinded_prod.columns

Out[231]: ['participant_id',
 'ingest_filename',
 'region',
 'country',
 'site_id',
 'randomization_date',
 'apoe4_carrier',
 'trial_assignment',
 'weight',
 'disposition']

In [0]:
#https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html

# PySpark for pandas



In [0]:
# Display the Schema of the DataFrame
print(registry.printSchema())

root
-- subject_event_crf_id: string (nullable = true)
-- user_create_id: string (nullable = true)
-- user_created_username: string (nullable = true)
-- user_created_fullname: string (nullable = true)
-- ts_create: string (nullable = true)
-- user_last_modify_id: string (nullable = true)
-- user_last_modify_username: string (nullable = true)
-- user_last_modify_fullname: string (nullable = true)
-- ts_last_modify: string (nullable = true)
-- dd_version_id: string (nullable = true)
-- dd_version_label: string (nullable = true)
-- dd_revision_id: string (nullable = true)
-- site_id: string (nullable = true)
-- site_code: string (nullable = true)
-- site_initials: string (nullable = true)
-- site_label: string (nullable = true)
-- subject_id: string (nullable = true)
-- subject_code: string (nullable = true)
-- subject_label: string (nullable = true)
-- event_id: string (nullable = true)
-- event_code: string (nullable = true)
-- event_label: string (nullable = true)
-- event_is_unscheduled: string (nullable = true)
-- language_code: string (nullable = true)
-- language_label: string (nullable = true)
-- has_unresolved_error: string (nullable = true)
-- is_locked: string (nullable = true)
-- user_last_lock_id: string (nullable = true)
-- user_last_lock_username: string (nullable = true)
-- user_last_lock_fullname: string (nullable = true)
-- ts_last_lock: string (nullable = true)
-- monitor_review: string (nullable = true)
-- monitor_review_label: string (nullable = true)
-- monitor_review_location: string (nullable = true)
-- monitor_review_location_label: string (nullable = true)
-- user_last_monitor_review_id: string (nullable = true)
-- user_last_monitor_review_username: string (nullable = true)
-- user_last_monitor_review_fullname: string (nullable = true)
-- ts_last_monitor_review: string (nullable = true)
-- dd_field_name: string (nullable = true)
-- dd_revision_field_id: string (nullable = true)
-- dd_revision_field_value: string (nullable = true)
-- dd_revision_field_translated_value: string (nullable = true)
-- dd_revision_field_is_missing: string (nullable = true)
-- dd_revision_field_is_phi: string (nullable = true)
-- dd_revision_field_label: string (nullable = true)
-- dd_revision_field_unit: string (nullable = true)
-- ingest_filename: string (nullable = true)

None

In [0]:
#Show the Head of the DataFrame

#registry.show(5)

#registry.take(5)


In [0]:
#Select Columns from the DataFrame

#registry.select('subject_label')



In [0]:
#Show the Statistics of the py DataFrame

#registry.describe().show()

In [0]:

# 
# *****************************************************************************
# subjects1: “subject_label” variable for studysum table or registry table
# 
# 
# non-missing exam date2: from registry table where variable dd_field.name= “examdate” 
# and variable dd_revision_field.translated_value has non-missing value
# 
# 
# exam date3: from registry table where variable dd_field.name= “examdate”
# 
# visit4: “event.label” from registry table 
# 
# 
# disposition status5: from registry table where variable dd_field.name= “sdstatus” and 
# variable dd_revision_field.translated_value = “Never randomized”
# 
# subjects6: “PARTICIPANT ID” variable for a3blinded_prod table or a45blinded_prod table
# ******************************************************************************


In [0]:

# 1.	Consented: Unique subjects1 from either registry table OR studysum table
# Screen Fail pre-SV1: Unique subjects1 from studysum table that DO NOT have any non-missing exam date2 from the registry table.
# 

#import os
#print(os.getcwd())

#os.listdir('/databricks/driver')

In [0]:
import pandas as pd 

# Convert Spark Nested Struct DataFrame to Pandas

# https://sparkbyexamples.com/pyspark/convert-pyspark-dataframe-to-pandas/
# selected varables for the demonstration
col1 = ['subject_label']
df1 = registry.select(col1)
# df1.show()
df2 = studysum.select(col1)
df1 = df1.toPandas()
df2 = df2.toPandas()
df3 = [df1, df2]
Consented = pd.concat(df3).drop_duplicates()
Consented



#type(df1)
#df1 <- registry %>% select(`Subject Label`) 
#df2 <- studysum %>% select(`Subject Label`)
#df3 <- union(df1, df2) %>% distinct()
#df3
#names(df3)

,subject_label
0,AHD324-11514
1,AHD041-10968
10,AHD041-11693
19,AHD305-11807
26,AHD705-11047
...,...
18641,AHD349-12132
19712,AHD382-11973
23535,AHD709-11017
25868,AHD307-11194


In [0]:
#Consented.write.saveAsTable(Consented) 

#Consented.DataFrameWriter.saveAsTable(Consented)

In [0]:
# PySpark

# https://towardsdatascience.com/pandas-to-pyspark-in-6-examples-bd8ab825d389
# Screen_Fail_pre_SV1

from pyspark.sql import functions as F

df1 = registry.filter(
    (F.col('dd_field_name') == 'examdate') &
    (F.col('dd_revision_field_translated_value') != '')
)



df2 = studysum 
col1 = ['subject_label']
df1 = registry.select(col1).toPandas()
#df1.show()
df3 = df2.select(col1).toPandas()

#df4 <-df3[!(df3$`Subject Label` %in% df1$`Subject Label`), ] 


#array = [1, 2, 3]
#df3.filter(df3.isin(df1) == False)


#df3[df3.subject_label isin (df1.subject_label), ]

Screen_Fail_pre_SV1= df3[~df3.subject_label.isin(df1.subject_label)].drop_duplicates()



Screen_Fail_pre_SV1
# .filter(registry.dd_field_name == "examdate" & registry.dd_revision_field_translated_value).collect()



,subject_label
10334,AHD336-11261
14706,AHD356-11110
14727,AHD356-11308
15924,AHD098-11884
16703,AHD137-11458
17039,AHD336-11383
17102,AHD336-11513
17165,AHD336-11574
17186,AHD303-11130
17234,AHD303-11687


In [0]:

# 2.	SV1
# Completed: Unique subjects1 who have completed the visit4 “Screening - Stage 1” with
#a non-missing exam date3 in the registry table.

# Pending: subjects1 who have visit4 “Screening - Stage 1” when
#the subjects1 have the maximum exam date3 AND the subjects1 are not present in the studysum table 


# Screen Fail: subjects1 who have visit4 “Screening - Stage 1” when the subjects1 have
# the maximum exam date3 AND the subjects1 are present in the studysum table with disposition status5 ‘Never randomized’
# 
# \

#Consented

In [0]:
from pyspark.sql import functions as F

df1 = registry.filter(
    (F.col('dd_field_name') == 'examdate') & (F.col('event_label') == 'Screening - Stage 1') & 
    (F.col('dd_revision_field_translated_value') != '')
)

SV1_Completed = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()

SV1_Completed



,subject_label
0,AHD324-11514
1,AHD041-10968
10,AHD041-11693
19,AHD305-11807
26,AHD705-11047
...,...
60059,AHD099-11433
60398,AHD381-11410
60414,AHD381-11415
60586,AHD130-11438


In [0]:
from pyspark.sql import functions as F

df1 = registry.filter(
    (F.col('dd_field_name') == 'examdate') & (F.col('event_label') == 'Screening - Stage 1') & 
    (F.col('dd_revision_field_translated_value') != '')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate'))

df2 = Consented[Consented.subject_label.isin(df1.toPandas().
                                                       subject_label)].drop_duplicates()


df3 = studysum.toPandas()

df4 = Consented[~Consented.subject_label.isin(df3.subject_label)]

df5 = pd.merge(df2, df4,on='subject_label')


SV1_Pending = df5

SV1_Pending

,subject_label
0,AHD324-11514
1,AHD041-11693
2,AHD305-11807
3,AHD705-11047
4,AHD712-11913
...,...
668,AHD705-11269
669,AHD013-11288
670,AHD190-11229
671,AHD309-11345


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 1') &
                      (F.col('dd_revision_field_translated_value') != '')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate'))

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()




df3 = studysum.filter((F.col('dd_field_name') == 'sdstatus') & (F.col('dd_revision_field_translated_value') == 'Never randomized') ).select('subject_label')




df4 = Consented[Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()




df5 = pd.merge(df2, df4, on = 'subject_label').drop_duplicates()

SV1_Screen_Fail = df5


SV1_Screen_Fail


,subject_label
0,AHD041-10968
1,AHD803-11377
2,AHD041-10966
3,AHD041-11549
4,AHD375-11524
...,...
1432,AHD099-11433
1433,AHD381-11410
1434,AHD381-11415
1435,AHD130-11438


In [0]:
# 
# 
# 
# 3.	SV2
# Completed: Unique subjects1 who have completed the visit4 “Screening - Stage 2” with a non-missing exam date3 in the registry table.
# Pending: subjects1 who have visit4 “Screening - Stage 2” when the subjects1 have the maximum exam date3 AND the subjects1 are not present in the studysum table 
# Screen Fail: subjects1 who have visit4 “Screening - Stage 2” when the subjects1 have the maximum exam date3 AND the subjects1 are present in the studysum table 
# with disposition status5 ‘Never randomized’
# 
# 
# 
# 
# 
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 2') &
                      (F.col('dd_revision_field_translated_value') != ''))


df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()

SV2_Completed = df2

SV2_Completed


,subject_label
0,AHD324-11514
1,AHD041-10968
10,AHD041-11693
26,AHD705-11047
35,AHD712-11913
...,...
53224,AHD177-11179
53254,AHD710-11201
53544,AHD703-11212
53893,AHD033-11209


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 2') &
                      (F.col('dd_revision_field_translated_value') != '')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3= studysum.toPandas()


df4 = Consented[~Consented.subject_label.isin(df3.subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how = 'inner',on = 'subject_label')


SV2_Pending = df5.drop_duplicates()

SV2_Pending




,subject_label
0,AHD324-11514
1,AHD041-11693
2,AHD705-11047
3,AHD712-11913
4,AHD137-10028
...,...
425,AHD356-11854
426,AHD158-11423
427,AHD705-10782
428,AHD705-11000


In [0]:

from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 2') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.filter((F.col('dd_field_name') == 'sdstatus') &
                      (F.col('dd_revision_field_translated_value') == 'Never randomized')).dropDuplicates()


df4 = Consented[Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV2_Screen_Fail = df5.drop_duplicates()

SV2_Screen_Fail






,subject_label
0,AHD041-10968
1,AHD803-11377
2,AHD041-10966
3,AHD041-11549
4,AHD375-11524
...,...
906,AHD177-11179
907,AHD710-11201
908,AHD703-11212
909,AHD033-11209


In [0]:
# 
# 
# 
# 4.	SV3
# Completed: Unique subjects1 who have completed the visit4 “Screening - Stage 3” with a non-missing exam date3 in the registry table.
# Pending: subjects1 who have visit4 “Screening - Stage 3” when the subjects1 have the maximum exam date3 AND the subjects1 are not present in the studysum table 
# Screen Fail: subjects1 who have visit4 “Screening - Stage 3 when the subjects1 have the maximum exam date3 AND the subjects1 are present in the studysum table with disposition status5 ‘Never randomized’
# 
# 
# 
# 



from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 3') &
                      (F.col('dd_revision_field_translated_value') != ' ')).dropDuplicates()

SV3_Completed = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()


SV3_Completed




,subject_label
0,AHD324-11514
10,AHD041-11693
26,AHD705-11047
35,AHD712-11913
44,AHD137-10028
...,...
53224,AHD177-11179
53254,AHD710-11201
53544,AHD703-11212
53893,AHD033-11209


In [0]:

from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 3') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum

df4 = Consented[~Consented.subject_label.isin(df3.toPandas().subject_label)]

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV3_Pending = df5.drop_duplicates()

SV3_Pending



,subject_label
0,AHD324-11514
1,AHD041-11693
2,AHD705-11047
3,AHD712-11913
4,AHD137-10028
...,...
329,AHD023-10964
330,AHD158-11423
331,AHD705-10782
332,AHD705-11000


In [0]:

from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 3') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.filter((F.col('dd_field_name') == 'sdstatus') &
                      (F.col('dd_revision_field_translated_value') == 'Never randomized')).dropDuplicates()


df4 = Consented[Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV3_Screen_Fail = df5.drop_duplicates()

SV3_Screen_Fail


,subject_label
0,AHD803-11377
1,AHD041-10966
2,AHD041-11549
3,AHD375-11524
4,AHD322-11678
...,...
880,AHD177-11179
881,AHD710-11201
882,AHD703-11212
883,AHD033-11209


In [0]:

# 
# 5.	SV4
# Completed: Unique subjects1 who have completed the visit4 “Screening - Stage 4” with a non-missing exam date3 in the registry table.
# Pending: subjects1 who have visit4 “Screening - Stage 4” when the subjects1 have the maximum exam date3 AND the subjects1 are not present in the studysum table 
# Screen Fail: subjects1 who have visit4 “Screening - Stage 4 when the subjects1 have the maximum exam date3 AND the subjects1 are present in the studysum table with disposition status5 ‘Never randomized’
# 




from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 4') &
                      (F.col('dd_revision_field_translated_value') != ' ')).dropDuplicates()

SV4_Completed = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()


SV4_Completed












,subject_label
0,AHD324-11514
26,AHD705-11047
44,AHD137-10028
132,AHD803-10612
148,AHD305-10427
...,...
44767,AHD027-10843
45749,AHD023-10885
48644,AHD023-11015
50151,AHD033-11051


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 4') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.dropDuplicates()


df4 = Consented[~Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV4_Pending  = df5.drop_duplicates()

SV4_Pending 

,subject_label
0,AHD324-11514
1,AHD705-11047
2,AHD137-10028
3,AHD803-10612
4,AHD305-10427
...,...
269,AHD023-11361
270,AHD381-11519
271,AHD029-11397
272,AHD109-10844


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 4') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.filter((F.col('dd_field_name') == 'sdstatus') &
                      (F.col('dd_revision_field_translated_value') == 'Never randomized')).dropDuplicates()


df4 = Consented[Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV4_Screen_Fail = df5.drop_duplicates()

SV4_Screen_Fail


,subject_label
0,AHD382-11166
1,AHD309-10025
2,AHD129-10047
3,AHD137-10058
4,AHD094-10074
5,AHD137-10075
6,AHD316-10114
7,AHD378-10122
8,AHD129-10119
9,AHD141-10152


In [0]:


# 
# 6.	SV5
# Completed: Unique subjects1 who have completed the visit4 “Screening - Stage 5” with a non-missing exam date3 in the registry table.
# Pending: subjects1 who have visit4 “Screening - Stage 5” when the subjects1 have the maximum exam date3 AND the subjects1 are not present in the studysum table 
# Screen Fail: subjects1 who have visit4 “Screening - Stage 5 when the subjects1 have the maximum exam date3 AND the subjects1 are present in the studysum table with disposition status5 ‘Never randomized’
# 
from pyspark.sql import functions as F
df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 5') &
                      (F.col('dd_revision_field_translated_value') != ' ')).dropDuplicates()
SV5_Completed = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()
SV5_Completed

,subject_label
0,AHD324-11514
26,AHD705-11047
44,AHD137-10028
132,AHD803-10612
148,AHD305-10427
...,...
27130,AHD701-11029
27313,AHD177-10881
29439,AHD109-10844
29523,AHD129-10769


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 5') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.dropDuplicates()


df4 = Consented[~Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV5_Pending = df5.drop_duplicates()

SV5_Pending

,subject_label
0,AHD324-11514
1,AHD705-11047
2,AHD137-10028
3,AHD803-10612
4,AHD305-10427
...,...
226,AHD098-10811
227,AHD381-11379
228,AHD023-11361
229,AHD109-10844


In [0]:
from pyspark.sql import functions as F


df1 = registry.filter((F.col('dd_field_name') == 'examdate') &
                      (F.col('event_label') == 'Screening - Stage 5') &
                      (F.col('dd_revision_field_translated_value') != ' ')).groupBy('subject_label').agg(F.max('dd_revision_field_translated_value').alias('max_examdate')).dropDuplicates()

df2 = Consented[Consented.subject_label.isin(df1.toPandas().subject_label)].drop_duplicates()



df3 = studysum.filter((F.col('dd_field_name') == 'sdstatus') &
                      (F.col('dd_revision_field_translated_value') == 'Never randomized')).dropDuplicates()


df4 = Consented[Consented.subject_label.isin(df3.toPandas().subject_label)].drop_duplicates()

df5 =  pd.merge(df2, df4, how='inner', on = 'subject_label')


SV5_Screen_Fail = df5.drop_duplicates()

SV5_Screen_Fail

,subject_label
0,AHD137-10058
1,AHD129-10119
2,AHD332-10221
3,AHD332-10297
4,AHD332-10320
5,AHD324-10358
6,AHD027-10462
7,AHD701-11029
8,AHD191-11118


In [0]:

# 
# 7.	Randomized: Unique subjects6 from either a3blinded_prod table or a45blinded_prod table
# Pending Randomization: (<SV5>.Completed minus <SV5>.Screen Fail minus Randomized)
# 
# Note: <SV5> data from step 6
# 
# 
# 
import pandas as pd 

# Convert Spark Nested Struct DataFrame to Pandas

# https://sparkbyexamples.com/pyspark/convert-pyspark-dataframe-to-pandas/
# selected varables for the demonstration
col1 = ['participant_id']
df1 = a3blinded_prod.select(col1)
#df1.show()
df2 = a45blinded_prod.select(col1)
df1 = df1.toPandas()
df2 = df2.toPandas()
df3 = [df1, df2]
Randomized = pd.concat(df3).drop_duplicates()
#Randomized.rename(columns={"subject_label": "subject_label"})

Randomized.rename({'participant_id': 'subject_label'}, axis=1, inplace=True)
Randomized



,subject_label
0,941-10115
1,094-10080
2,094-10323
3,941-10313
4,041-10428
...,...
144,802-10928
145,802-10984
146,810-11776
147,803-11223


In [0]:
# df1 <- SV5_Completed[!(SV5_Completed$`Subject Label` %in% SV5_Screen_Fail$`Subject Label` )]

from pyspark.sql import functions as F

PendingRandomization = SV5_Completed[~SV5_Completed.subject_label.isin(SV5_Screen_Fail.subject_label)]

PendingRandomization

,subject_label,Category
0,AHD324-11514,SV5_Completed
26,AHD705-11047,SV5_Completed
44,AHD137-10028,SV5_Completed
132,AHD803-10612,SV5_Completed
148,AHD305-10427,SV5_Completed
...,...,...
25497,AHD381-11379,SV5_Completed
27064,AHD023-11361,SV5_Completed
27313,AHD177-10881,SV5_Completed
29439,AHD109-10844,SV5_Completed


In [0]:
# show all the varaibles 

%whos 

Variable Type Data/Info
---------------------------------------------------------
Category list n=215
Consented DataFrame subject_label\n0 <...>\n[2141 rows x 1 columns]
Consented1 list n=2141
DateType type <class 'pyspark.sql.types.DateType'>
F module <module 'pyspark.sql.func<...>yspark/sql/functions.py'>
FloatType type <class 'pyspark.sql.types.FloatType'>
IntegerType type <class 'pyspark.sql.types.IntegerType'>
PandasUDFType type <class 'pyspark.sql.panda<...>functions.PandasUDFType'>
PendingRandomization DataFrame subject_label\n0 <...>n\n[236 rows x 1 columns]
Randomized DataFrame participant_id\n0 <...>n\n[215 rows x 1 columns]
SQLContext type <class 'pyspark.sql.context.SQLContext'>
SV1_Completed DataFrame subject_label\n0 <...>\n[2115 rows x 1 columns]
SV1_Pending DataFrame subject_label\n0 A<...>n\n[673 rows x 1 columns]
SV1_Screen_Fail DataFrame subject_label\n0 <...>\n[1437 rows x 1 columns]
SV2_Completed DataFrame subject_label\n0 <...>\n[1346 rows x 1 columns]
SV2_Pending DataFrame subject_label\n0 A<...>n\n[430 rows x 1 columns]
SV2_Screen_Fail DataFrame subject_label\n0 A<...>n\n[911 rows x 1 columns]
SV3_Completed DataFrame subject_label\n0 <...>\n[1224 rows x 1 columns]
SV3_Pending DataFrame subject_label\n0 A<...>n\n[334 rows x 1 columns]
SV3_Screen_Fail DataFrame subject_label\n0 A<...>n\n[885 rows x 1 columns]
SV4_Completed DataFrame subject_label\n0 <...>n\n[320 rows x 1 columns]
SV4_Pending DataFrame subject_label\n0 A<...>n\n[274 rows x 1 columns]
SV4_Screen_Fail DataFrame subject_label\n0 AHD<...>3-11051\n40 AHD381-11097
SV5_Completed DataFrame subject_label\n0 <...>n\n[245 rows x 1 columns]
SV5_Pending DataFrame subject_label\n0 A<...>n\n[231 rows x 1 columns]
SV5_Screen_Fail DataFrame subject_label\n0 AHD13<...>01-11029\n8 AHD191-11118
SV_all DataFrame subject_label <...>n[13086 rows x 5 columns]
Screen_Fail_pre_SV1 DataFrame subject_label\n1033<...>1194\n27871 AHD382-11301
SparkConf type <class 'pyspark.conf.SparkConf'>
SparkContext type <class 'pyspark.context.SparkContext'>
SparkDF_Consented DataFrame DataFrame[subject_label: <...>string, Category: string]
SparkDF_PendingRandomization DataFrame DataFrame[subject_label: string]
SparkDF_Randomized DataFrame DataFrame[participant_id: string]
SparkDF_SV1_Completed DataFrame DataFrame[subject_label: string]
SparkDF_SV1_Pending DataFrame DataFrame[subject_label: string]
SparkDF_SV1_Screen_Fail DataFrame DataFrame[subject_label: string]
SparkDF_SV2_Completed DataFrame DataFrame[subject_label: string]
SparkDF_SV2_Pending DataFrame DataFrame[subject_label: string]
SparkDF_SV2_Screen_Fail DataFrame DataFrame[subject_label: string]
SparkDF_SV3_Completed DataFrame DataFrame[subject_label: string]
SparkDF_SV3_Pending DataFrame DataFrame[subject_label: string]
SparkDF_SV3_Screen_Fail DataFrame DataFrame[subject_label: string]
SparkDF_SV4_Completed DataFrame DataFrame[subject_label: string]
SparkDF_SV4_Pending DataFrame DataFrame[subject_label: string]
SparkDF_SV4_Screen_Fail DataFrame DataFrame[subject_label: string]
SparkDF_SV5_Completed DataFrame DataFrame[subject_label: string]
SparkDF_SV5_Screen_Fail DataFrame DataFrame[subject_label: string]
SparkDF_Screen_Fail_pre_SV1 DataFrame DataFrame[subject_label: string]
SparkSession type <class 'pyspark.sql.session.SparkSession'>
StringType type <class 'pyspark.sql.types.StringType'>
StructField type <class 'pyspark.sql.types.StructField'>
StructType type <class 'pyspark.sql.types.StructType'>
Window type <class 'pyspark.sql.window.Window'>
a3blinded_prod DataFrame DataFrame[participant_id:<...>ing, disposition: string]
a45blinded_prod DataFrame DataFrame[participant_id:<...>ing, disposition: string]
abs function <function _create_functio<...>als>._ at 0x7fe66ac38a70>
acos function <function _create_functio<...>als>._ at 0x7fe66ac3e170>
add_months function <function add_months at 0x7fe66ac42f80>
approxCountDistinct function <function approxCountDistinct at 0x7fe66ac41050>
approx_count_distinct function <function ap

In [0]:

%sql 
use enrollment_abba303
-- use the database to write my tables 

-- three ways to write table in PySpark

--https://towardsdatascience.com/3-ways-to-create-tables-with-apache-spark-32aed0f355ab


In [0]:
type(Consented)

Consented

,subject_label
0,AHD324-11514
1,AHD041-10968
10,AHD041-11693
19,AHD305-11807
26,AHD705-11047
...,...
18641,AHD349-12132
19712,AHD382-11973
23535,AHD709-11017
25868,AHD307-11194


In [0]:
Category1 = ['Consented']*len(Consented)
Consented['Category'] = Category1

Category2 = ['Screen_Fail_pre_SV1']*len(Screen_Fail_pre_SV1)
Screen_Fail_pre_SV1['Category'] = Category2




Category3 = ['SV1_Completed']*len(SV1_Completed)
SV1_Completed['Category'] = Category3


Category4 = ['SV1_Pending']*len(SV1_Pending)
SV1_Pending['Category'] = Category4

Category5 = ['SV1_Screen_Fail']*len(SV1_Screen_Fail)
SV1_Screen_Fail['Category'] = Category5


Category6 = ['SV2_Completed']*len(SV2_Completed)
SV2_Completed['Category'] = Category6


Category7 = ['SV2_Pending']*len(SV2_Pending)
SV2_Pending['Category'] = Category7

Category8 = ['SV2_Screen_Fail']*len(SV2_Screen_Fail)
SV2_Screen_Fail['Category'] = Category8



Category9 = ['SV3_Completed']*len(SV3_Completed)
SV3_Completed['Category'] = Category9


Category10 = ['SV3_Pending']*len(SV3_Pending)
SV3_Pending['Category'] = Category10

Category11 = ['SV3_Screen_Fail']*len(SV3_Screen_Fail)
SV3_Screen_Fail['Category'] = Category11



Category12 = ['SV4_Completed']*len(SV4_Completed)
SV4_Completed['Category'] = Category12


Category13 = ['SV4_Pending']*len(SV4_Pending)
SV4_Pending['Category'] = Category13

Category14 = ['SV4_Screen_Fail']*len(SV4_Screen_Fail)
SV4_Screen_Fail['Category'] = Category14


Category15 = ['SV5_Completed']*len(SV5_Completed)
SV5_Completed['Category'] = Category15


Category16 = ['SV5_Pending']*len(SV5_Pending)
SV5_Pending['Category'] = Category16

Category17 = ['SV5_Screen_Fail']*len(SV5_Screen_Fail)
SV5_Screen_Fail['Category'] = Category17

Category18 = ['PendingRandomization']*len(PendingRandomization)
PendingRandomization['Category'] = Category18

Category19 = ['Randomized']*len(Randomized)
Randomized['Category'] = Category19












/local_disk0/tmp/1636060193367-0/PythonShell.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 from RichCompleter import RichCompleter

In [0]:
SV_all = pd.concat([Consented,
Screen_Fail_pre_SV1,
SV1_Completed,
SV1_Pending,
SV1_Screen_Fail,
SV2_Completed,
SV2_Pending,
SV2_Screen_Fail,
SV3_Completed,
SV3_Pending,
SV3_Screen_Fail,
SV4_Completed,
SV4_Pending,
SV4_Screen_Fail,
SV5_Completed,
SV5_Pending,
SV5_Screen_Fail,
Randomized,
PendingRandomization])


#SparkDF_SV_all=spark.createDataFrame(SV_all) 

In [0]:
import pyspark

from pyspark.sql import SparkSession

import pandas as pd

spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

#pdDF = pd.read_csv("samp.csv")

from pyspark.sql.types import *

mySchema = StructType([StructField('subject_label', StringType(), True)\

                       ,StructField('Category', StringType(), True)])

df_SV_all = spark.createDataFrame(SV_all, schema=mySchema)


df_SV_all.display()


subject_label,Category
AHD324-11514,Consented
AHD041-10968,Consented
AHD041-11693,Consented
AHD305-11807,Consented
AHD705-11047,Consented
AHD712-11913,Consented
AHD137-10028,Consented
AHD803-11366,Consented
AHD803-11377,Consented
AHD023-11890,Consented


In [0]:
%sql 
use enrollment_abba303



In [0]:


df_SV_all_tb = spark.sql("select * from enrollment_abba303.df_SV_all")
df_SV_all_tb.display()







subject_label,Category
AHD324-11514,Consented
AHD041-10968,Consented
AHD041-11693,Consented
AHD305-11807,Consented
AHD705-11047,Consented
AHD712-11913,Consented
AHD137-10028,Consented
AHD803-11366,Consented
AHD803-11377,Consented
AHD023-11890,Consented


In [0]:
import pyspark

from pyspark.sql.functions import to_date, col

df= (spark.read.format('csv')\
.option('header', 'True')\
.option('inferSchema', 'true')\
.load('dbfs:/FileStore/tables/SV_all.csv'))




--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-230896138177681> in <module> 
 6 . option ( 'header' , 'True' ) \ 
 7 . option ( 'inferSchema' , 'true' ) \ 
 ----> 8 .load('dbfs:/FileStore/tables/SV_all.csv'))
 9 
 10 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 176 self . options ( ** options ) 
 177 if isinstance ( path , basestring ) : 
 --> 178 return self . _df ( self . _jreader . load ( path ) ) 
 179 elif path is not None : 
 180 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o10231.load.
: com.immuta.spark.exceptions.ImmutaSecurityException: You may not access raw data directly.
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.$anonfun$validatePlan$1(Unknown Source)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.$anonfun$validatePlan$1$adapted(Unknown Source)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreach(TreeNode.scala:182)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.validatePlan(Unknown Source)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.validatePlan$(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.validatePlan(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.$anonfun$apply$1(Unknown Source)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.immuta.spark.util.ProfilerUtil$.profile(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.apply(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$42(CheckAnalysis.scala:675)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$42$adapted(CheckAnalysis.scala:675)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:675)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:206)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apa

In [0]:
import pyspark

from pyspark.sql.functions import to_date, col

df = spark.read.format('csv').option('header', 'True').option('inferSchema', 'true').load('dbfs:/FileStore/tables/sMRI_unnormalised_Screening_Failure.csv')








--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-230896138177682> in <module> 
 3 from pyspark . sql . functions import to_date , col
 4 
 ----> 5 df = spark . read . format ( 'csv' ) . option ( 'header' , 'True' ) . option ( 'inferSchema' , 'true' ) . load ( 'dbfs:/FileStore/tables/sMRI_unnormalised_Screening_Failure.csv' ) 
 6 
 7 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 176 self . options ( ** options ) 
 177 if isinstance ( path , basestring ) : 
 --> 178 return self . _df ( self . _jreader . load ( path ) ) 
 179 elif path is not None : 
 180 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o10288.load.
: com.immuta.spark.exceptions.ImmutaSecurityException: You may not access raw data directly.
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.$anonfun$validatePlan$1(Unknown Source)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.$anonfun$validatePlan$1$adapted(Unknown Source)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreach(TreeNode.scala:182)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.validatePlan(Unknown Source)
	at com.immuta.spark.databricks.DatabricksImmutaTableResolver.validatePlan$(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.validatePlan(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.$anonfun$apply$1(Unknown Source)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.immuta.spark.util.ProfilerUtil$.profile(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.apply(Unknown Source)
	at com.immuta.spark.ImmutaCheckRule.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$42(CheckAnalysis.scala:675)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$42$adapted(CheckAnalysis.scala:675)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:675)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:206)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:845)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(Quer

In [0]:
df_SV_all.write.mode("overwrite").saveAsTable("df_SV_all")

In [0]:
#OTHER DATA SOURCES FOR MANAGED & UNMANAGED TABLES

#sparkDF.createOrReplaceTempView('df_SV_all') #source_2



In [0]:
#%sql
#CREATE TABLE IF NOT EXISTS SV_all_tb2 AS 
#SELECT * FROM df_SV_all

In [0]:
#spark.sql(“CREATE TABLE IF NOT EXISTS salesTable_manag2 AS SELECT * FROM df_final_View”)

In [0]:
#METHOD 1 - Managed Table
#sparkDF.write.mode("overwrite").saveAsTable("PendingRandomization_tb1")

In [0]:
#spark.catalog.listTables()

In [0]:
#sparkDF.repartition(2).write.mode("overwrite").save("/FileStore/PendingRandomization_View.parquet") #source_3

In [0]:
%whos 

In [0]:
# show the paths

%fs ls / 

In [0]:
# read data from the path

#import pyspark
#from pyspark.sql.functions import to_date, col

#LOAD CSV FILE - INCLUDES 5M ROWS
#df = (spark.read.format('csv')
#           .option('header', 'True')
#           .option("inferSchema", "true")
#           .load('/FileStore/tables/SV_all.csv'))

In [0]:

#%sql 
#use enrollment_abba303





In [0]:
#%sql 

#create table enrollment_abba303.test as 
#SELECT 'testdata' as dummyv



In [0]:
#spark.sql("CREATE DATABASE learn_spark_db")


In [0]:
#studysum.show()